# **SELECCIÓN DE CARACTERÍSTICAS**

In [1]:
#IMPORTACIÓN DE LIBRERÍAS

# En este apartado se realizará la importación de todas las librerías 
# a ser ocupadas para la creación del modelo como tersor flow , pandas ,sklearn 
# y numpy

import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import matplotlib.pyplot as plt


In [2]:
#CONEXIÓN A DRIVE Y LECTURA DEL DATASET

# El entorno de trabajo es colab por lo que es necesario importar el drive si nuestros archivos se 
# encuentran en este.

from google.colab import drive
drive.mount('/content/drive')

# Mediante la librería de pandas mandamos a buscar el archivo a la carpeta contenida en el drive
# una vez encontrado se colocaran tags a las columnas del dataframe

#El primer dataset se usa para la implementación del modelo de random forest, para la importancia de las características.
df1 = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

#Al primer dataset se le harán algunas modificaciones, por lo que se define un segundo dataset, con el mismo archivo. 
#Este segundo dataset se lo usa para la implementación del modelo
df = pd.read_excel("/content/drive/MyDrive/Colab/ResultadosCompleto.xlsx")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#INFORMACIÓN DEL DATASET
# Con el comando "info()", se obtiene el tipo de datos de las columnas y el número total de registros
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   edad          2210 non-null   int64 
 1   genero        2210 non-null   object
 2   personalidad  2210 non-null   object
 3   estilo        2210 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 69.2+ KB


In [4]:
df1=df1[df1['edad']>16]
df1=df1[df1['edad']<31]
df1

,edad,genero,personalidad,estilo
0,25,Masculino,Confiable,5
1,28,Femenino,Animado,4
2,18,Femenino,Animado,6
4,21,Masculino,Confiable,3
5,20,Masculino,Animado,10
...,...,...,...,...
2205,26,Femenino,Animado,10
2206,29,Femenino,Animado,1
2207,22,Masculino,Extravertido,8
2208,22,Masculino,Responsable,4


In [5]:
#VERIFICACIÓN DE VALORES NULOS EN EL DATASET

#Se debe comprobar que en el dataset no existan valores nulos, para que en los siguientes procesos, no se muestren resultados inconsistentes
check_for_nan = df1.isnull().values.any()
print (check_for_nan)



False


In [6]:
#TRANSFORMACIÓN DE CARACTERÍSTICAS CATEGÓRICAS A NUMÉRICAS PARA LA SELECCIÓN DE CARACTERÍSTICAS
#CON RANDOM FOREST

# Se realiza la respectiva transformación de los datos para que el modelo pueda entenderlos,
# como se puede ver en el siguiente algoritmo, todos los datos de las columnas de género y 
# personalidad se les asignará un número reemplazando los valores no numéricos

df1['genero'] = df1['genero'].map({'Masculino': 0,
                                 'Femenino': 1})

df1['personalidad'] = df1['personalidad'].map({'Confiable': 0,
                                           'Extravertido': 1,
                                           'Animado': 2,
                                           'Responsable': 3,
                                           'Serio': 4})

In [7]:
df1

,edad,genero,personalidad,estilo
0,25,0,0,5
1,28,1,2,4
2,18,1,2,6
4,21,0,0,3
5,20,0,2,10
...,...,...,...,...
2205,26,1,2,10
2206,29,1,2,1
2207,22,0,1,8
2208,22,0,3,4


In [8]:
#DECLARACIÓN DE VARIABLES TEMPORALES PARA X E Y PARA LA SELECCIÓN DE CARACTERÍSTICAS

#Para las variables en X, seleccionamos todas las variables del dataset, excepto la variable que tendremos de salida: el estilo. 

X_inicial = df1.iloc[:,0:3]
y_inicial = df1.iloc[:,-1]

In [9]:
#El algoritmo de random forest de clasificación, contiene un comando que nos permite calcular el rango de importancia de cada variable.
#Así pues, según el resultado que se obtenga en este top, se comprueba si se está seleccionando correctamente las características. 

#En este paso, se realiza un entrenamiento parcial con las variables sin procesar. 
#Allí se consideran las variables independientes y la variable dependiente, tando de entrenamiento como de testeo. 
#Con este proceso, lo que se busca es listar a las características en orden ascendente, desde la más importante. 
#Debido a que solo se tienen 3 datos de entrada, el array consta de 3 variables. 


RF = RandomForestClassifier(max_depth=7, n_estimators=100)

df1 = pd.DataFrame()
RF.fit(X_inicial, y_inicial)
y_pred_RF= RF.predict(X_inicial)
cols = ['Machine Learning Classification Method',
        'Train Accuracy', 
        'Test Accuracy', 
        "Top 1 Feature Predictor",
        "Top 2 Feature Predictor",
        "Top 3 Feature Predictor"]

df1.loc['Machine Learning Classification Method', 
       'Details'] = 'Random Forest Classifier'
df1.loc['Train Accuracy', 'Details'] = RF.score(X_inicial, y_inicial) 



feature_importances = X_inicial.columns[np.argsort(RF.feature_importances_)][-3:]
df1.loc['Top 1 Feature Predictor', 'Details'] = feature_importances[2]
df1.loc['Top 2 Feature Predictor', 'Details'] = feature_importances[1]
df1.loc['Top 3 Feature Predictor', 'Details'] = feature_importances[0]

display(df1)

,Details
Machine Learning Classification Method,Random Forest Classifier
Train Accuracy,0.422244
Top 1 Feature Predictor,personalidad
Top 2 Feature Predictor,edad
Top 3 Feature Predictor,genero


# **AHORA QUE SE TIENE UNA LISTA DE IMPORTANCIA DE LAS CARACTERÍSTICAS SE PROCEDE A LAS DEBIDAS TRANSFORMACIONES**

In [10]:
#TRANSFORMACIÓN DE EDAD CON LA TÉCNICA DE BINNING

#Se coloca el intervalo de edades que está permitido en el dataset (de 17 a 30 años, según el objetivo de análisis del proyecto)
#donde se dividirán en 5 rangos, que estarán etiquetados con los nombres de la variable "labels"

bins = np.linspace(16,31,6)
labels = ['0.1', '0.2', '0.3','0.4','0.5']
df['edad'] = pd.cut(df['edad'], bins, labels=labels)

print(bins)
print(df)

[16. 19. 22. 25. 28. 31.]
     edad     genero  personalidad  estilo
0     0.3  Masculino     Confiable       5
1     0.4   Femenino       Animado       4
2     0.1   Femenino       Animado       6
3     NaN  Masculino  Extravertido       3
4     0.2  Masculino     Confiable       3
...   ...        ...           ...     ...
2205  0.4   Femenino       Animado      10
2206  0.5   Femenino       Animado       1
2207  0.2  Masculino  Extravertido       8
2208  0.2  Masculino   Responsable       4
2209  0.1  Masculino     Confiable       7

[2210 rows x 4 columns]


In [11]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS LUEGO DE LA APLICACIÓN DE TÉCNICA DE BINNING

#Al utilizar binning para colocar los datos en rangos, se puede observar que el tipo de dato se cambió a categórico
#Esto indica que más adelante, se deberá realizar nuevamente una transformación para que se vuelva un dato de tipo numérico.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2210 entries, 0 to 2209
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   edad          2115 non-null   category
 1   genero        2210 non-null   object  
 2   personalidad  2210 non-null   object  
 3   estilo        2210 non-null   int64   
dtypes: category(1), int64(1), object(2)
memory usage: 54.3+ KB


In [12]:
df

,edad,genero,personalidad,estilo
0,0.3,Masculino,Confiable,5
1,0.4,Femenino,Animado,4
2,0.1,Femenino,Animado,6
3,NaN,Masculino,Extravertido,3
4,0.2,Masculino,Confiable,3
...,...,...,...,...
2205,0.4,Femenino,Animado,10
2206,0.5,Femenino,Animado,1
2207,0.2,Masculino,Extravertido,8
2208,0.2,Masculino,Responsable,4


In [13]:
#VERIFICACIÓN DE DATOS NULOS 

#Es importante que luego de hacer una modificación de alguna variable en el dataset, se vuelva a comprobar que no existan datos nulos.
check_for_nan = df.isnull().values.any()
print (check_for_nan)

#Con el siguiente comando, se imprimen la cantidad de nulos que existen y en qué columnas se encuentran. 
#Solo en la edad existen datos nulos, ya que existían personas que no tenían la edad dentro del rango permitido; por tanto, no se les otorgó un
#intervalo con el proceso de binning y pasaron a ser datos nulos.
df.isnull().sum()

True


edad            95
genero           0
personalidad     0
estilo           0
dtype: int64

In [14]:
#COMO SI EXISTEN VALORES NULOS, SE LOS DEBE ELIMINAR

#Como los datos nulos, son de las edades que no corresponden al intervalo escogido para la muestra, se deben eliminar. 
#De esta manera, de los 2209 registros que se tenía en un inicio, se disminuyeron a 1956.
df = df[~df['edad'].isnull()].copy()
df

,edad,genero,personalidad,estilo
0,0.3,Masculino,Confiable,5
1,0.4,Femenino,Animado,4
2,0.1,Femenino,Animado,6
4,0.2,Masculino,Confiable,3
5,0.2,Masculino,Animado,10
...,...,...,...,...
2205,0.4,Femenino,Animado,10
2206,0.5,Femenino,Animado,1
2207,0.2,Masculino,Extravertido,8
2208,0.2,Masculino,Responsable,4


In [15]:
#NORMALIZACIÓN DE LA VARIABLE DE SALIDA (ESTILO)

#Como la variable de salida, no se encuentra en el mismo rango que el resto de los datos, se procede a realizar la normalización de esta variable.
#Para ello, se define una variable que contenga nuestros datos de salida
y_for_norm = df.iloc[:,-1]

#Y se realiza la normalización de Mínimo y máximo, para otorgarle valores al estilo de entre 0 y 1.
def minmax_norm(variable):
    return (variable - variable.min()) / ( variable.max() - variable.min())

y_norm = minmax_norm(y_for_norm)

print(y_norm)

0       0.444444
1       0.333333
2       0.555556
4       0.222222
5       1.000000
          ...   
2205    1.000000
2206    0.000000
2207    0.777778
2208    0.333333
2209    0.666667
Name: estilo, Length: 2115, dtype: float64


In [16]:
#TÉCNICA ONE_HOT_ENCONDE PARA LA TRANSFORMACIÓN DE VARIABLES CATEGÓRICAS A NUMÉRICAS
one_hot_genero = pd.get_dummies(df.genero ,prefix='genero')
one_hot_Personalidad = pd.get_dummies(df.personalidad ,prefix='personalidad')

#CONCATENACIÓN DE TODAS LAS VARIABLES TRANSFORMADAS

#Ahora que ya se encuentran todas las variables transformadas, se procede a unirlas en una sola tabla.
d = pd.concat([df.edad,one_hot_genero, one_hot_Personalidad,y_norm], axis=1)
d

,edad,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio,estilo
0,0.3,0,1,0,1,0,0,0,0.444444
1,0.4,1,0,1,0,0,0,0,0.333333
2,0.1,1,0,1,0,0,0,0,0.555556
4,0.2,0,1,0,1,0,0,0,0.222222
5,0.2,0,1,1,0,0,0,0,1.000000
...,...,...,...,...,...,...,...,...,...
2205,0.4,1,0,1,0,0,0,0,1.000000
2206,0.5,1,0,1,0,0,0,0,0.000000
2207,0.2,0,1,0,0,1,0,0,0.777778
2208,0.2,0,1,0,0,0,1,0,0.333333


In [17]:
#VERIFICACIÓN DE LOS TIPOS DE DATOS QUE SE TRANSFORMARON
#Luego de realizar todas las transformaciones, obtenemos los tipos de datos que se encuentran en el dataset, para confirmar
#que ya no se necesiten más transformaciones
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2115 entries, 0 to 2209
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   edad                       2115 non-null   category
 1   genero_Femenino            2115 non-null   uint8   
 2   genero_Masculino           2115 non-null   uint8   
 3   personalidad_Animado       2115 non-null   uint8   
 4   personalidad_Confiable     2115 non-null   uint8   
 5   personalidad_Extravertido  2115 non-null   uint8   
 6   personalidad_Responsable   2115 non-null   uint8   
 7   personalidad_Serio         2115 non-null   uint8   
 8   estilo                     2115 non-null   float64 
dtypes: category(1), float64(1), uint8(7)
memory usage: 49.8 KB


In [18]:
#Definidos ya todos los valores, se exporta el dataset con el que se trabajará en los siguientes procesos. 
d.to_excel("DataFrameTransformado.xlsx")

#En new_data se almacena el dataset que se acaba de exportar, para que, mediante esta variable, se llame al nuevo dataframe
new_data = pd.read_excel("DataFrameTransformado.xlsx")

In [19]:
#DEFINICIÓN DE VARIABLES DE ENTRADA Y LA VARIABLE DE SALIDA

#Ya teniendo todos los datos transformados, se puede definir "x" e "y" para trabajar con ellas, en el proceso posterior e implementarlas ya en el modelo
X = new_data.iloc[:,0:9]
y = new_data.iloc[:,-1].values
X


,Unnamed: 0,edad,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio
0,0,0.3,0,1,0,1,0,0,0
1,1,0.4,1,0,1,0,0,0,0
2,2,0.1,1,0,1,0,0,0,0
3,4,0.2,0,1,0,1,0,0,0
4,5,0.2,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
2110,2205,0.4,1,0,1,0,0,0,0
2111,2206,0.5,1,0,1,0,0,0,0
2112,2207,0.2,0,1,0,0,1,0,0
2113,2208,0.2,0,1,0,0,0,1,0


In [20]:
X = X.drop(['Unnamed: 0'], axis=1)
X

,edad,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio
0,0.3,0,1,0,1,0,0,0
1,0.4,1,0,1,0,0,0,0
2,0.1,1,0,1,0,0,0,0
3,0.2,0,1,0,1,0,0,0
4,0.2,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2110,0.4,1,0,1,0,0,0,0
2111,0.5,1,0,1,0,0,0,0
2112,0.2,0,1,0,0,1,0,0
2113,0.2,0,1,0,0,0,1,0


In [21]:
X

,edad,genero_Femenino,genero_Masculino,personalidad_Animado,personalidad_Confiable,personalidad_Extravertido,personalidad_Responsable,personalidad_Serio
0,0.3,0,1,0,1,0,0,0
1,0.4,1,0,1,0,0,0,0
2,0.1,1,0,1,0,0,0,0
3,0.2,0,1,0,1,0,0,0
4,0.2,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2110,0.4,1,0,1,0,0,0,0
2111,0.5,1,0,1,0,0,0,0
2112,0.2,0,1,0,0,1,0,0
2113,0.2,0,1,0,0,0,1,0


In [22]:
#DIVISIÓN DEL DATASET PARA LOS DATOS DE ENTRENAMIENTO Y LOS DE TESTEO

#Finalmente, se dividen los datos para que sean 70% para el entrenamiento y el 30% para el testeo.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y, 
                                                    test_size=0.3, random_state=100)

In [23]:
# Crea tu primer MLP en Keras
from keras.models import Sequential
from keras.layers import Dense
import numpy
# Fija las semillas aleatorias para la reproducibilidad
numpy.random.seed(7)
# carga los datos

# crea el modelo
model = Sequential()
model.add(Dense(8, input_dim=8, activation='sigmoid'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compila el modelo
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Ajusta el modelo
model.fit(X_train, y_train, epochs=1500, batch_size=10)
# evalua el modelo
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6905 - accuracy: 0.1047
Epoch 2/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6860 - accuracy: 0.1608
Epoch 3/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6818 - accuracy: 0.1736
Epoch 4/1500
148/148 [==============================] - 0s 3ms/step - loss: 0.6775 - accuracy: 0.1743
Epoch 5/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6731 - accuracy: 0.1736
Epoch 6/1500
148/148 [==============================] - 0s 3ms/step - loss: 0.6692 - accuracy: 0.1716
Epoch 7/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6665 - accuracy: 0.1709
Epoch 8/1500
148/148 [==============================] - 1s 4ms/step - loss: 0.6642 - accuracy: 0.1655
Epoch 9/1500
148/148 [==============================] - 0s 2ms/step - loss: 0.6626 - accuracy: 0.1682
Epoch 10/1500
148/148 [==============================] - 0s 3ms/step - loss: 0.661

## ¿Qué modelo de ML/DL/DM usted aplicará a su proyecto para ser resuelto?

El modelo de machin learning a utilizar son las redes ¿ neuronales, en base a la investigación bibliográfica realizada de otros proyectos este es uno de los modelos más usados para la predicción de personalidad y moda. Además de que obtiene muy buenos resultados con el tiempo suficiente de entrenamiento y el ajuste de los hyperparametros correcto.

## ¿Con qué modelos comparará ese modelo y por qué?

Para la comparación de resultados , el modelo de redes neuronales será comparado con el modelo SVM este constará con 3 kernels, cada uno de estos kernels contendrá diferentes formas de entrenamiento de los datos, los kernels a utilizar son polinomial, Radial Basis Function (RBF) y sigmoidal.

## ¿Cuáles son los hyperparameters que utilizará para cada modelo?

### Redes neuronales

Los hyperparameters seleccionados para el ajuste del modelo fueron los siguientes:”Input_shape” El número de entradas que tomara el modelo, esto dependerá del número total de datos que tendrá la columna. El “epochs” el cual representara el numero total de veces que se enviaran los datos a la red, otro parámetro seleccionado es el “Alpha” el cual representa el número máximo de cambio de pesos de las neuronas con la finalidad que no sean tan brusco y aprenda de forma correcta, también es necesario definir el “Optimizer” el es el algoritmo de optimización que usara el modelo, como penúltimo hyperparametro se define el ”loss” el cual indicara el porcentaje de datos perdidos en la red neuronal, y el ultimo hyperparametro seleccionado son la métricas a utilizar, el cual constara de la métrica de accuracy


### SVM


En este modelo se debe establecer los diferentes núcleos que se van a evaluar, entre los más comunes están los kernels polinomial, Radial Basis Function (RBF) y sigmoidal. El siguiente hyperparametro es la regularización “C” el cual representara el error de clasificación. Otro de los parámetros utilizado es “Gamma”. Este define la distancia en el cálculo de la línea de separación aceptable.


## ¿Qué métricas utilizará para evaluar su modelo?

Las métricas que se evaluaran para el modelo será el “acurracy” el cual nos permitirá medir cual es el porcentaje en el que el modelo acertó en la predicción y la segunda métrica a utilizar será “precisión” con el cual obtendremos el porcentaje de valores reales que pudo predecir el modelo

## Interpretacion final

Para el tratamiento de los datos se aplicaron diferentes técnicas, entre las cuales se encuentra el binning de la edad, con esta se presente reducir el rango de valores que contiene la columna edad del dataframe. Luego se aplicó one_hot a todo el dataset excepto a la columna de estilos. El one_hot debe tomar una sola columna a la vez por lo que al final es necesario unir cada uno de estos resultados en un dataframe. Después de realizar todo el tratamiento de los datos, se verifico si existían datos nulos, y se eliminaron. Debido al tipo de datos que obtenemos al aplicar el respectivo tratamiento, se pretende realizar la transformación de estos a enteros, para poder enviárselo al modelo sin ningún tipo de error o inconveniente

## Link a GitHub

https://github.com/Mystarioc/-NRC_8888-YandryMonta-o_Evaluaci-nEnLineaUnidad3.git